In [2]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_counterfactual")

No config specified, defaulting to: amazon_counterfactual/en
Found cached dataset amazon_counterfactual (/home/yuetian/.cache/huggingface/datasets/mteb___amazon_counterfactual/en/1.0.0/3b6ae425288b4ec49ffa16445fa595593e9c00e210da62c9cf576ad2acfe4a8b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Convert the 'train' and 'test' subsets into pandas DataFrames
train_df = dataset['train'].to_pandas()
test_df = dataset['test'].to_pandas()

In [4]:
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")

In [5]:
test_df['label'].value_counts()

0    539
1    131
Name: label, dtype: int64

In [34]:
import pandas as pd
import os
import json

def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data


def generate_data(threshold, dataframe, dataset):
    mask = []
    class_num = 0

    # Iterate over class directories until no more are found
    while True:
        class_dir = os.path.join('..', '..', 'score', dataset+"_eval", f'class{class_num}')

        # If the directory doesn't exist, break the loop
        if not os.path.exists(class_dir):
            break

        file_path_class = os.path.join(class_dir, f'mask_class{class_num}.json')
        loaded_predicted_class = load_json(file_path_class)
        mask_class = [score[0] for score in loaded_predicted_class if score[2] < threshold]
        mask.extend(mask_class)
        

        class_num += 1

    # Merge the masks and remove duplicates
    mask = list(set(mask))

    # Filter the DataFrame based on the index list
    filtered_df = dataframe.loc[mask].dropna()

    # Write the filtered DataFrame to a new CSV file
    data_dir = os.path.join(f"{threshold}_test.csv")
    filtered_df.to_csv(data_dir, index=False)

    return data_dir

def generate_threshold_data(thresholds: list, ground_truth: pd.DataFrame, dataset: str) -> list:
    """Generates data for each threshold."""
    filenames = []
    for threshold in thresholds:
        filename = generate_data(threshold, ground_truth, dataset)
        filenames.append(filename)
    return filenames


generate_threshold_data([0.2, 0.4, 0.6, 0.8, 1], pd.read_csv("./test.csv"), "mteb_counterfactual")

['0.2_test.csv', '0.4_test.csv', '0.6_test.csv', '0.8_test.csv', '1_test.csv']

In [35]:
test_df = pd.read_csv("1_test.csv")

In [36]:
test_df['label'].value_counts()

0    538
1    130
Name: label, dtype: int64

In [38]:
from transformers import AutoModel

model_name = "distilbert-base-uncased"  # Replace with your desired model name
model = AutoModel.from_pretrained(model_name)

num_params = model.num_parameters()
print(f"{model_name} has {num_params} parameters.")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


distilbert-base-uncased has 66362880 parameters.


In [39]:
from transformers import AutoModel

model_name = "microsoft/deberta-v3-large"  # Replace with your desired model name
model = AutoModel.from_pretrained(model_name)

num_params = model.num_parameters()
print(f"{model_name} has {num_params} parameters.")


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


microsoft/deberta-v3-large has 434012160 parameters.
